In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp
import csv  # ★変更点: csvモジュールをインポート

# --- 設定 ---
# ★変更点: CSVデータ用の保存先パスを新たに設定
DATA_PATH_CSV = os.path.join(os.getcwd(), 'MP_Data_CSV') 
#actions = np.array(['deadpoint', 'dyno', 'cross_move'])
actions = np.array(['heel_hook', 'deadpoint', 'dyno','cross_move'])
no_videos = 0
sequence_length = 180

# --- MediaPipe Poseモデルの準備 ---
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# 取得したい姿勢ランドマークのインデックスを指定
DESIRED_POSE_LANDMARKS = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

def multiple_detection(image, model):
    # この関数は変更ありません
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # この関数は変更ありません
    image_height, image_width, _ = image.shape

    if results.pose_landmarks:
        # 点の描画
        for idx in DESIRED_POSE_LANDMARKS:
            if idx < len(results.pose_landmarks.landmark):
                landmark = results.pose_landmarks.landmark[idx]
                if landmark.visibility < 0.1:
                     continue
                cx = int(landmark.x * image_width)
                cy = int(landmark.y * image_height)
                cv2.circle(image, (cx, cy), 3, (0, 0, 255), cv2.FILLED)

        # 線の描画
        if mp_pose.POSE_CONNECTIONS:
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                if start_idx in DESIRED_POSE_LANDMARKS and end_idx in DESIRED_POSE_LANDMARKS:
                    if start_idx < len(results.pose_landmarks.landmark) and \
                       end_idx < len(results.pose_landmarks.landmark):

                        start_landmark = results.pose_landmarks.landmark[start_idx]
                        end_landmark = results.pose_landmarks.landmark[end_idx]

                        if start_landmark.visibility < 0.1 or end_landmark.visibility < 0.1:
                             continue

                        start_point = (int(start_landmark.x * image_width), int(start_landmark.y * image_height))
                        end_point = (int(end_landmark.x * image_width), int(end_landmark.y * image_height))
                        cv2.line(image, start_point, end_point, (51, 255, 51), 2)

# ★変更点: JSON出力用の extract_keypoints 関数は不要なため削除しました。

# --- データの収集 ---
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

with mp_pose.Pose(min_detection_confidence=0.1, min_tracking_confidence=0.1, model_complexity=2) as pose_model:
    # ★変更点: CSVのヘッダー行を動的に作成
    header = ['frame']
    for landmark_id in DESIRED_POSE_LANDMARKS:
        header += [f'{landmark_id}_x', f'{landmark_id}_y', f'{landmark_id}_z', f'{landmark_id}_visibility']
    
    for action in actions:
        # ★変更点: CSV用のアクションごとのフォルダを作成
        action_path = os.path.join(DATA_PATH_CSV, action)
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            video_file_path = os.path.join(r'C:\Users\admin\Downloads\senteivideo(1)', f'{action}{video_num}.mp4')
            
            cap = cv2.VideoCapture(video_file_path)
            # 動画ファイルが開けない場合はスキップ
            if not cap.isOpened():
                print(f"警告: 動画ファイルが開けませんでした: {video_file_path}")
                continue

            # ★変更点: 動画ごとにCSVファイルを開き、書き込み準備
            csv_file_path = os.path.join(action_path, f'{video_num}.csv')
            print(f'処理中の動画: {video_file_path}')
            
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow(header)  # ヘッダーを書き込む

                for frame_num in range(sequence_length):
                    ret, frame = cap.read()
                    if not ret:
                        print(f"情報: 動画 {video_file_path} が {sequence_length} フレーム未満で終了しました。収集済みフレーム数: {frame_num}")
                        break

                    image, results = multiple_detection(frame, pose_model)
                    draw_styled_landmarks(image, results)

                    cv2.putText(image, f'Collecting frames for {action} - Video {video_num} - Frame {frame_num + 1}/{sequence_length}',
                                (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                    # ★変更点: キーポイントを抽出し、CSVの1行として書き込む
                    row = [frame_num]
                    if results.pose_landmarks:
                        for landmark_id in DESIRED_POSE_LANDMARKS:
                            res = results.pose_landmarks.landmark[landmark_id]
                            row.extend([res.x, res.y, res.z, res.visibility])
                    else:
                        # 骨格が検出されなかった場合、空欄で埋める
                        num_keypoints_data = len(DESIRED_POSE_LANDMARKS) * 4
                        row.extend([''] * num_keypoints_data)
                    
                    csv_writer.writerow(row)

                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                
            cap.release()
            print(f"データ保存完了: {csv_file_path}")

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

処理中の動画: C:\Users\admin\Downloads\senteivideo(1)\heel_hook1.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\heel_hook\1.csv
処理中の動画: C:\Users\admin\Downloads\senteivideo(1)\heel_hook2.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\heel_hook\2.csv
処理中の動画: C:\Users\admin\Downloads\senteivideo(1)\heel_hook3.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\heel_hook\3.csv
処理中の動画: C:\Users\admin\Downloads\senteivideo(1)\heel_hook4.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\heel_hook\4.csv
処理中の動画: C:\Users\admin\Downloads\senteivideo(1)\heel_hook5.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\heel_hook\5.csv
処理中の動画: C:\Users\admin\Downloads\senteivideo(1)\heel_hook6.mp4
デー